In [95]:
#別ファイルのpyは一度消さないと
#更新しても再インポートされない。
#一度リロードさせるための処理
import sys
try:
    del sys.modules['dataframe']
except Exception as e:
    pass

import dataframe
nikkei225 = dataframe.DataFrame()

def create_date_picker():
    import ipywidgets as widgets
    date_picker = widgets.DatePicker(description='日付を選択', disabled=False)
    display( widgets.interactive(on_date_change, date=date_picker) )

def on_date_change( date ):
    if "-" in str(date):
        global nikkei225
        nikkei225.fetch( str(date).replace( "-", "." ) )
        nikkei225.display()
        print( "Loading Completed")
        
if __name__ == "__main__":
    create_date_picker()


interactive(children=(DatePicker(value=None, description='日付を選択'), Output()), _dom_classes=('widget-interact',…

In [99]:
selected_stocks = []

def minmax( l, margin = 4 ):
    min = l.min()
    max = l.max()
    return min-margin, max+margin

import matplotlib.pyplot as plt
def plot_slope_prices( selections, start):
    y_min = sys.maxsize
    y_max = -sys.maxsize
    xs = []
    ys = []
    ls = []
    
    global selected_stocks
    for s in selected_stocks:
        x,y = s.plot_change_xy()
        xs.append( x )
        ys.append( y )
        ls.append( str(s.code) )
        min, max = minmax( y )
        if min < y_min: 
            y_min = min
        if max > y_max:
            y_max = max
    
    diff = 5.0
    fig = plt.figure(figsize=(16,8))
    ax = fig.add_subplot(111)
    rect = plt.Rectangle((start-diff,y_min),diff,y_max-y_min,fc="#c0c0c0")
    ax.add_patch(rect)
    for i in range( len(xs) ):
        plt.plot(xs[i],ys[i],label=ls[i])
    plt.legend( loc = "upper left")
    plt.grid( True )
    plt.ylim( y_min, y_max )
        
    plt.show()


def create_sel_menu( combo_count ):
    opts = []
    nikkei225.get_combo_info_list( combo_count )
    for l in nikkei225.combo_info_list:
        opts.append(l["timestamp"])
    select_menu = widgets.Select(options=opts)
    display( widgets.interactive(on_selection_change, selection=select_menu) )
    


def on_selection_change( selection ):
    global selected_stocks 
    for l in nikkei225.combo_info_list:
        if l["timestamp"] == selection:
            start_idx = nikkei225.find_timestamps( selection )
            selected_stocks = []
            lbls = ""
            for s in l["stocks"]:
                selected_stocks.append( s )
                lbls += s.label+ "(x " + str(int(s.combos[start_idx])) +") "
            plot_slope_prices( selected_stocks, start_idx )
            print( lbls )
            break
            
if __name__ == "__main__":
    create_sel_menu( 5 )


interactive(children=(Select(description='selection', options=('09:25:00', '09:30:00', '09:40:00', '09:45:00',…